# Criticality Searches

Criticality searches, or the adjustment of some free parameter of your problem (e.g. boron concentation, control rod insertion point), is a routine part of neutronics. In this example, we'll perform a critical search based on fuel enrichment. OpenMC performs this criticality search using the methodology from Price and Roskoff (https://linkinghub.elsevier.com/retrieve/pii/S014919702300166X), a GRsecant (generalized regressive secant) method. This method takes into account the statistical uncertainty in $k$ and also dynamically adjusts the number of batches run (using more batches as one gets closer to criticality).

In [ ]:
import openmc

model = openmc.Model()

In [ ]:
uo2 = openmc.Material()
uo2.add_element('U', 1.0, enrichment=3.5)
uo2.add_element('O', 2.0)
uo2.set_density('g/cm3', 10.97)

zirconium = openmc.Material()
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.55)



In [ ]:
box = openmc.model.RectangularPrism(1.2, 1.2, boundary_type='reflective')
top = openmc.ZPlane(z0=300.0, boundary_type='vacuum')
bottom = openmc.ZPlane(z0=0.0, boundary_type='vacuum')

fuel_surf = openmc.ZCylinder(r=0.39)
clad_surf = openmc.ZCylinder(r=0.45)

fuel = openmc.Cell(region=-fuel_surf & -box, fill=uo2)
clad = openmc.Cell(region=+fuel_surf & -clad_surf & -box, fill=zirconium)
h2o = openmc.Cell(region=+clad_surf & -box, fill=water)

model.geometry = openmc.Geometry(openmc.Universe(cells=[fuel, clad, h2o]))

model.geometry.root_universe.plot(width=(2, 2))

In [ ]:
model.settings.particles = 10000
model.settings.inactive = 10
model.settings.batches = 50

When running a criticality search, we can specify the desired maximum uncertainty in $k$ with `sigma_final` and the tolerance to which we want $k$ to match unity with `k_tol`.

## Geometry

Criticality searches work for any change to a mutable object - in this next example, we'll take a sphere of uranium enclosed in a metal aluminum shell of 1 cm thickness, and determine the critical radius.

In [ ]:
model = openmc.Model()

fuel = openmc.Material()
fuel.add_nuclide('U235', 1.0)
fuel.set_density('g/cm3', 19.1)

al = openmc.Material()
al.add_element('Al', 1.0)
al.set_density('g/cm3', 2.7)
dr = 1.0



In order to change the radius of the fuel sphere, we want to modify the `in_sphere.r` parameter. Note that we have to also update the `out_sphere.r` parameter! When we use `r=in_sphere.r + dr`, that only applies a value (not a reference to `in_sphere.r`), so it also needs to be updated by us.

In [ ]:
model.settings.particles = 10000
model.settings.inactive = 10
model.settings.batches = 50

